<a href="https://colab.research.google.com/github/solidgose/solidgose_experiments/blob/master/bert_tpu_tweet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with BERT, TPUs, and engineered features on Google Colab
## Overview and Objective
This Colab notebook demonstrates the ability to conduct fast text classification using the Tensor Processing Units (TPUs) provided through Google Colab.

Using two publicly-downloadable text databases: the fivethirtyeight.com's list of Internet Research Agency's (IRA) associated tweets during the 2016 General Election in the United States, and George Washington University's database of tweets identified with the 2016 election, a BERT model will be fine-tuned and trained to determine if a tweet from these dataset is associated with the IRA (or not).

This notebook uses the BERT model checkpoints, not the TensorFlow Hub-hosted BERT models. This is because the TensorFlow Hub model uses a tf.Placeholder/feed_dict that reduces performance on TPUs and is incompatible for conducting inference. This notebook instead uses tf.Dataset with TFRecords.

For now, this notebook only uses the English BERT models and English examples. More languages may be added later.

## Disclaimer and License
This notebook is intended for demonstration and research purposes only, and is not intended to be used in any production or actual predictive capacity. Also, Google Colab is intended for research; any computationally intensive forks of this code should probably be operated on a paid GCS instance. 

Where code has been copied or altered from other sources, the code will be commented to indicate the original source of the code. 

The twitter data downloaded by this notebook contains expletives, sexual references, and other strong language. URLs contained in tweets may link to web pages with malware, pornography, or other nefarious content. (Or, a link to a perfectly normal web page!) These links are not accessed or followed in this notebook. Nevertheless, proceed with caution! 

Code utilized in this notebook is adopted from the Apache License 2.0, and thus this notebook falls under the same license.

This notebook is not intended to serve in any production or actual capacity - this is for experimentation and demonstration purposes only.

## Prerequisites
**Google Colab**: This notebook is designed to run on Google Colab; in Google Colab make sure your Runtime -> Change Runtime Session is set to *Python 3* and that your hardware accelerator is *TPU*. Otherwise, this code won't run. This notebook has not been tested in other environments.

**Google Cloud Storage**: A Google Cloud Compute storage account is required to operate this notebook. The account is required because TPUs can only access data from a Google Storage Container (gs://) and not a local file system. Information on establishing a Google Cloud account is here: (URL Goes Here). Typical use of this notebook will result in very minimal costs (< USD 1.00).

**Twitter Developer Account**: While the fivethirtyeight.com tweets are available for download without a license, the GWU tweets must be downloaded from Twitter's API. This requires the use of a Twitter Developer Account - the "free" tier of this service is sufficient to run this demonstration. The public and private "consumer" and "app" keys are needed to pull this data. More information on this is available here: (info here)

**Questions or Feedback**: Post an issue to https://github.com/solidgose


In [0]:
import pickle
import pprint
import json
import re
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tqdm import tqdm
from os import listdir
from os.path import isfile, join, exists
import os
import requests
import sys
from glob import glob
from itertools import chain
import time
import math
import numpy as np
import urllib.request
!pip install twitter
from twitter import *
!pip install namegenerator
import namegenerator
!pip install bert-tensorflow
import bert
import ipywidgets as widgets
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling


# About Settings and Defaults


### GCS Bucket
A GS bucket is a "gs://bucket" (where bucket is your bucket) location. If the bucket is not present under your GCS project ID, a bucket will be created with this code.

### tf_checkpoint_root
This is the root location where new fine-tuned and trained BERT model checkpoints will be located. 

### data_loc
Where the source data for this notebook will be downloaded.

### data_cache
Location for temporary files created for use by the TPU and other procedures.

### project_id
This is your Google Compute Services project-id. A GCS billable project with a project ID is required to utilize this workbook.

### Twitter API IDs
These IDs are required for the *twitter* pypi package to download the tweets for this exercise. A Twitter developer account is needed to complete this information. 

### What happens when you run this code block
1. The global constants needed to set file locations for the notebook are created.

2. This Google Colab instance will log into your Google Cloud Compute API, confirm your identity, and display an authentication token needed to re-enter back into this notebook. The token is needed for the notebook to access your Google Cloud Storage.

In [0]:
#@title Settings and defaults
gcs_bucket = '<bucket name here>' #@param {type:"string"}
tf_checkpoint_root = 'tf_checkpoint_root' #@param {type:"string"}
#tf_hub_cache = 'tf_hub_cache' #@param {type:"string"}
data_loc = 'data_loc' #@param {type:"string"}
data_cache = 'data_cache' #@param {type:"string"}
project_id = '<input project id>' #@param {type:"string"}
consumer_twitter_api = '<input consumer twitter api>' #@param {type:"string"} 
consumer_twitter_api_private = '<input consumer twitter api private>' #@param {type:"string"} 
access_api = '<input access api>' #@param {type:"string"}
access_api_private = '<input access api private>' #@param {type:"string"}


BUCKET_LOC = f'gs://{gcs_bucket}/'
TF_CHECKPOINT_ROOT = f'gs://{gcs_bucket}/{tf_checkpoint_root}'
DATA_DIR = f'gs://{gcs_bucket}/{data_loc}'
#GS_TF_HUB_CACHE = f'gs://{gcs_bucket}/{tf_hub_cache}'
DATA_CACHE = f'gs://{gcs_bucket}/{data_cache}'
CONSUMER_TWITTER_API = consumer_twitter_api
CONSUMER_TWITTER_API_PRIVATE = consumer_twitter_api_private
ACCESS_TWITTER_API = access_api
ACCESS_TWITTER_API_PRIVATE = access_api_private

# Now credentials are set for all future sessions on this TPU.
!gcloud config set project {project_id}

from google.colab import auth
auth.authenticate_user()

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

print("All defaults successfully established")

label_list = [0,1]
use_TPU = True
random_seed = 42

In [0]:
# Create any buckets or bucket folders needed for code execution.

bucket_status = !gsutil ls {BUCKET_LOC}
if "AccessDeniedException" in bucket_status:
    !gsutil mb -p {project_id} {BUCKET_LOC}
    print(f'Created {BUCKET_LOC}')
else:
    print(f'{BUCKET_LOC} already exists.')
    
if not tf.io.gfile.exists(TF_CHECKPOINT_ROOT):
    print(f'Created {TF_CHECKPOINT_ROOT}')
    tf.gfile.MakeDirs(TF_CHECKPOINT_ROOT)
else:
    print(f'{TF_CHECKPOINT_ROOT} already exists.')
        
if not tf.io.gfile.exists(DATA_DIR):
    print(f'Created {DATA_DIR}')
    tf.gfile.MakeDirs(DATA_DIR)
else:
    print(f'{DATA_DIR} already exists.')
          
if not tf.io.gfile.exists(DATA_CACHE):
    print(f'Created {DATA_CACHE}')
    tf.gfile.MakeDirs(DATA_CACHE)
else:
    print(f'{DATA_CACHE} already exists.')

In [0]:
# As of Feb 2019, these are the BERT models available. This list may change as new models are introduced.

bert_model_urls = ["https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip",
                   "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip",
                   "https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip",
                   "https://storage.googleapis.com/bert_models/2018_10_18/cased_L-24_H-1024_A-16.zip"]

# If there are no sub-folders located in your tensorflow checkpoint root,
# this code will create one root to start with

list_of_tf_roots = tf.io.gfile.listdir(TF_CHECKPOINT_ROOT)
    
if len(list_of_tf_roots) == 0:
    print("There are no models present in your tf checkpoint root. \n Create a new model using this name or a name of your choice:")
    cur_name = namegenerator.gen()
    text_input = input(f'Enter a model name, or <ENTER> for {cur_name}:')
    if text_input is '':
        text_input=cur_name
    print(f'Creating folder for {text_input}')
    tf.gfile.MakeDirs(TF_CHECKPOINT_ROOT + '/' + text_input)
    print("Folder created")

The following code uses ipywidgets for dynamic selection of constants. (Google Colab does not support dynamic forms, unforuntately.) Running the code will allow you to select your BERT model and location for your model, for the code that will subsequently execute.

If you don't know which model to select, try 768/cased first.

In [0]:

print("Select a bert model to use")
bert_model_choice = widgets.Dropdown(options=bert_model_urls, value="https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip")
bert_model_choice



In [0]:
print("Select a location for the tensorflow model.")
tf_checkpoint_sub_dir = widgets.Dropdown(options=tf.io.gfile.listdir(TF_CHECKPOINT_ROOT), value=tf.io.gfile.listdir(TF_CHECKPOINT_ROOT)[0])
tf_checkpoint_sub_dir

In [0]:
print(f'Do you want to download the BERT model into {tf_checkpoint_sub_dir.value}?')
should_download = widgets.Dropdown(options=['Yes','No'], value='Yes')
should_download

In [0]:
def create_and_populate_checkpoint_folder(subfolder_name, bert_model_name_with_zip):
    try:
        
        # Check to see if there is a trained model in this folder, in this case,
        # halt and ask the user what to do.
        
        if tf.gfile.Exists(TF_CHECKPOINT_ROOT + '/' + subfolder_name +  'checkpoint'):
            print("There appears to be an existing checkpoint in this sub-folder.")
            print("Do you want to wipe this subfolder and insert a new BERT step 0 checkpoint?")
            answer = input("Type \'yes\' and <Enter> if you want to wipe. Type any other character to not wipe.'")
            if answer == "yes":
                !gsutil rm {TF_CHECKPOINT_ROOT + '/' + subfolder_name + '*'}
            else:
                print("Ending function early.")
                return True

        
        bert_file_name = bert_model_name_with_zip.split('/')[-1]
        print(f'Retrieving {bert_file_name}')

        # Check to see if this file has already been downloaded. If so,
        # skip the download step

        if not exists(bert_model_choice.value):
            urllib.request.urlretrieve(bert_model_choice.value, bert_file_name)
            print(f'{bert_file_name} successfully downloaded')
        else:
            print(f'{bert_file_name} already downloaded')
        # Re-unzip the bert model, just in case something funky happened
        !unzip -o {bert_file_name}
        !cd {bert_file_name.split('.')[0]};gsutil cp * {TF_CHECKPOINT_ROOT + '/' + subfolder_name}

        return True
    except Exception as e:
              print(e)
              return False

In [0]:
# If you chose to download the bert model, the following function will
# download and unzip the model in the designated location

if should_download.value == "Yes":
    create_and_populate_checkpoint_folder(subfolder_name = tf_checkpoint_sub_dir.value,
                                         bert_model_name_with_zip = bert_model_choice.value)


Now, we download the archive of tweets identified by fivethirtyeight.com as sourced from Russia's Internet Research Agency (IRA) during the 2016 election. This archive of tweets is served as a github repositry, which will be cloned and copied to the GCS bucket. 

Downloading the IRA tweets for the first time will take 5-10 minutes; the status may not update as the files are downloaded.

In [0]:


print(f'fivethirtyeight.com \'s tweets will be downloaded to {DATA_DIR}')
if tf.gfile.Exists(DATA_DIR +'/ru_tweets.h5'):
    print("Tweets already downloaded to GCS; mirroring copy onto local instance.")
    if not os.path.exists('ru_tweets.h5'):
        !gsutil cp {DATA_DIR}/ru_tweets.h5 ru_tweets.h5 
    ru_tweets = pd.read_hdf('ru_tweets.h5',key='ru_tweets')
    print('Tweets loaded into local instance and accessible as \'ru_tweets\'')
else:
    print("Cloning fivethirtyeight.com repo. This may take 5-10 minutes.")
    !git clone https://github.com/fivethirtyeight/russian-troll-tweets.git
    print("Tweet list successfully downloaded")
    cur_path = '/content/russian-troll-tweets/'
    ru_tweets = pd.concat([pd.read_csv(cur_path + f) if os.path.splitext(f)[1] == '.csv' else None for f in tqdm(listdir(cur_path))])
    # save as an hd5
    print("Caching tweets on GCS")
    print("Saving as h5")
    ru_tweets.to_hdf('ru_tweets.h5',key='ru_tweets')
    print("copying to gcs bucket")
    !gsutil cp ru_tweets.h5 {DATA_DIR}/ru_tweets.h5
    print("Operation completed.")
    

Next, we download a list of tweet ids, collected by George Washington University (GWU) and hosted on dataverse.harvard.edu. These tweet ids were collected by GWU for purposes of collecting all tweets relating to the 2016 election. Unlike the fivethirtyeight.com tweets that include content, GWU provides only the ID and not the actual content. The Twitter Developer keys will be used to download the tweet content directly from Twitter.

A collab instance does not have enough RAM/Disk Space to hold a large quantity of downloaded tweets. This code takes randomly samples 1/50 of each file.

In [0]:
print("Checking to see if we already have a cached version of this data.")
if tf.gfile.Exists(DATA_DIR +'/filter_list_pd.h5'):
    print("Data already downloaded and cached. Loadng from GCS and copying to local file system.")
    if not os.path.exists('filter_list_pd.h5'):
        !gsutil cp {DATA_DIR}/filter_list_pd.h5 filter_list_pd.h5  
    filter_list_pd = pd.read_hdf('filter_list_pd.h5',key='filter_list')
    print('tweet indices loaded')
else:    
    # Note: This code has been written in long-form, so that new different samples,
    # files, etc., can be input. Feel free to experiment as you wish.
    
    print("Downloading twitter IDs using pandas...")
    election_filter1_pd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PDI7IN/KAONZQ', header=None, sep = " ") 
    print("Election filter 1 downloaded")
    election_filter2_pd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PDI7IN/WMEGGS', header=None, sep = " ")
    print("Election filter 2 downloaded")
    election_filter3_pd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PDI7IN/LGVDZB', header=None, sep = " ")
    print("Election filter 3 downloaded")
    election_filter4_pd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PDI7IN/UQ7JRQ', header=None, sep = " ")
    print("Election filter 4 downloaded")
    election_filter5_pd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PDI7IN/XQYUAY', header=None, sep = " ")
    print("Election filter 5 downloaded")
    election_filter6_pd = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PDI7IN/ZJALBN', header=None, sep = " ")
    print("Election filter 6 downloaded")
    # now, we are only going to keep 1 million from each, since these files are too large.

    filter_list_pd = pd.concat([election_filter1_pd.sample(frac=(1/50), random_state=random_seed), 
                                election_filter2_pd.sample(frac=(1/50), random_state=random_seed),
                                election_filter3_pd.sample(frac=(1/50), random_state=random_seed), 
                                election_filter4_pd.sample(frac=(1/50), random_state=random_seed), 
                                election_filter5_pd.sample(frac=(1/50), random_state=random_seed), 
                                election_filter6_pd.sample(frac=(1/50), random_state=random_seed)]).reset_index()
    filter_list_pd.columns = ['index','tweet_id'] 
    filter_list_len = len(filter_list_pd)
    print(f'Loaded {filter_list_len} columns')
    filter_list_pd.info()
    filter_list_pd.to_hdf('filter_list_pd.h5',key='filter_list')
    print("copying to gcs bucket")
    !gsutil cp filter_list_pd.h5 {DATA_DIR}/filter_list_pd.h5
    print('Freeing up RAM...removing dataframes of original downloads')
    del election_filter1_pd
    del election_filter2_pd
    del election_filter3_pd
    del election_filter4_pd
    del election_filter5_pd
    del election_filter6_pd
    print("Twitter ID collection completed.")


The following operations are conducted to confirm our requests to the rate limits of Twitter's API, which allows 100 IDs per request, and 900 requests per 15 minutes. 

Of course, the more data you download, the larger the training set, and hypothetically, the greater likelihood that a random downloaded sample of tweets will be representative of the universe of potential tweets. (In other words, the more data, the less likely an overfit model will be fit.)

In [0]:
# divide the current filtered tweet list into buckets of 100 tweets
MAX_IDs_PER_REQUEST = 100
number_of_shards = math.ceil(len(filter_list_pd)/MAX_IDs_PER_REQUEST)
filter_lst_pd_only_indices = filter_list_pd[['tweet_id']]
filter_list_pd_sharded = np.array_split(filter_list_pd,number_of_shards)

In [0]:
# The data provided in the original files are integers.
# These integers need to be converted to strings, for purposes
# of calling the Twitter API
list_to_string = [list(map(str,f.tweet_id.tolist())) for f in filter_list_pd_sharded]
string_list_to_concat = [",".join(f) for f in list_to_string]
print('String conversion completed.')

In [0]:
MAX_REQUESTS_PER_15_MIN = 900
# now, divide string_list_to_concat into chunks of 900 shards, for submission to the twitter api
number_of_shards = math.ceil(len(string_list_to_concat)/MAX_REQUESTS_PER_15_MIN)
print(f'Created {number_of_shards} number_of_shards')
string_list_to_concat_sharded = np.array_split(string_list_to_concat,number_of_shards)

In [0]:
# Now, we instantiate the pypi Twitter API wrapper with the API keys collected.
try:
    t = Twitter(auth=OAuth(ACCESS_TWITTER_API, ACCESS_TWITTER_API_PRIVATE, CONSUMER_TWITTER_API, CONSUMER_TWITTER_API_PRIVATE))
except:
    except Exception as e:
        print("Sorry! You may not be using valid credentials. Please check the following error message and try again.")

In [0]:
#@title Select start shard and number to collect. (15 min per shard)
start_shard = 0 #@param {type:"integer"}
num_of_shards = 3 #@param {type:"integer"}


In [0]:
def countdown(t):
    while t > 0:
        sys.stdout.write('\rCountdown to next API pull : {}s'.format(t))
        t -= 1
        sys.stdout.flush()
        time.sleep(1)

In [0]:
def save_tweetdata_to_gs(in_json, i):
    pickle.dump(in_json, open(f'tweet100-{i}.pickle','wb'))
    !gsutil cp tweet100-{i}.pickle {DATA_DIR}/tweet100-{i}.pickle 

In [0]:
# Downloads the selected tweets, every 15 minutes
# Note: This will overwrite tweets with the same index number

for i in range(start_shard, start_shard + num_of_shards):
    ret_tweets = [t.statuses.lookup(_id=f) for f in tqdm(string_list_to_concat_sharded[i])]
    save_tweetdata_to_gs(ret_tweets, i)
    countdown(910)
    # Wait 15 min / 900 seconds until the next pull. Add 10 secs, just in case.

In [0]:
# Find all the available tweets saved on the gs bucket,
# copy them onto the local file system, and load them
# into RAM.

# This will overwrite any tweets still on the local file system.
# If you have already downloaded tweets using the above code,
# Then just run this code to restore what you have already downloaded.

!gsutil cp {DATA_DIR}/tweet100* .

# double-check that they have been loaded

!ls tweet100*


In [0]:
# Convert the downloaded tweets to pandas data frames

cur_tweets_files = glob('tweet100*')
print(cur_tweets_files)

tweet_dicts = [pickle.load(open(f,'rb')) for f in tqdm(cur_tweets_files)]
num_of_tweet_dicts = len(tweet_dicts)
print(f'{num_of_tweet_dicts} tweet files loaded')

# Unnest the dictionaries

In [0]:
def flatten_tweet_json(in_json):
    """
    Input is a json file from the twitter API.
    Returns a python dictionary that unnests the
    user-related content from the input json
    """
    
    
    ret_dict = {}
    cur_keys = in_json.keys()
    for k in cur_keys:
        if not type(in_json[k]) is dict:
            ret_dict.update({k:in_json[k]})
        else:
            for kk in in_json[k].keys():
                ret_dict.update({k + '.' + kk:in_json[k][kk]})
    return(ret_dict)
    

In [0]:
# Convert the tweet_dicts to a data frame, replacing our tweet_dicts to conserve memory
tweet_dicts = pd.DataFrame(list(map(flatten_tweet_json, tqdm(tweet_dicts))))

In [0]:
# The above steps are a bit time intensive. The following code
# allows you to backup and restore the data generated thus far.

def dump_prepared_tweets():
    pickle.dump(tweet_dicts,open('tweet_dicts_prepped.pickle','wb'))
    pickle.dump(ru_tweets, open('ru_tweets_prepped.pickle','wb'))
    !gsutil cp tweet_dicts_prepped.pickle {DATA_CACHE}/tweet_dicts_prepped.pickle 
    !gsutil cp ru_tweets_prepped.pickle {DATA_CACHE}/ru_tweets_prepped.pickle
    print('Prepped tweet dump and copy to GCS completed')
    return True


def load_prepared_tweets():
    if not os.path.exists('tweet_dicts_prepped.pickle'):
        print("Copying tweet_dicts_prepped.pickle from gcs")
        !gsutil cp {DATA_CACHE}/tweet_dicts_prepped.pickle .
    if not os.path.exists('ru_tweets_prepped.pickle'):
        !gsutil cp {DATA_CACHE}/ru_tweets_prepped.pickle .
        print("Copying ru_tweets_prepped.pickle from gc")
    tweet_dicts = pickle.load(open('tweet_dicts_prepped.pickle','rb'))
    ru_tweets = pickle.load(open('ru_tweets_prepped.pickle','rb'))    
    print("Prepared tweets loaded")
    return tweet_dicts, ru_tweets

In [0]:
# Uncomment these lines to dump and/or load the prepared tweets

# dump_prepared_tweets
tweet_dicts, ru_tweets = load_prepared_tweets()

The tweet content collected using the GWU list of twitter IDs may have tweets identified by fivethirtyeight.com as being authorited by IRA. Ideally, a training set should have an accurate a ground truth as possible: the list of GWU tweets should ideally not have any tweets from IRA.

For immediate purposes, we'll check the author name and text. Howeer, this step could probably be a bit more comprehensie.

In [0]:
# Collect a unique list of tweets written by the IRA, by screen name

ru_authors_list = [x.lower() for x in list(set(ru_tweets.author.tolist()))]
print(f'Number of unique screen names in the IRA dataset: {len(ru_authors_list)}')

In [0]:
# Conduct filtering
filtered_tweets_screen_names = [x.lower() for x in tweet_dicts['user.screen_name'].tolist()]
non_unique_screen_names_len = len(filtered_tweets_screen_names)
unique_screen_names_len = len(set(filtered_tweets_screen_names))
unique_screen_names = set(filtered_tweets_screen_names)
print(f'Found {non_unique_screen_names_len} non-unique screen names, and {len(unique_screen_names)} unique screen names in filtered tweets')

overlapping_screen_names = set(unique_screen_names) & set(ru_authors_list)
print(f'There are {len(overlapping_screen_names)} overlapping screen names')


In [0]:
# Now, check for exact text content of tweets between ru and filtered.
# It is likely that a russian tweet is in our list of filtered tweets

overlapping_text = set(tweet_dicts['text'].tolist()) & set(ru_tweets['content'].tolist())

# Remove the overlapping tweets from our filtered list

gwu_tweets = tweet_dicts[~tweet_dicts['text'].isin(ru_tweets['content'])]

# The IRA tweet database has non-English tweets
# For now, let's limit the IRA database to English tweets

ru_tweets = ru_tweets.loc[ru_tweets.language == 'English']

In [0]:
"""
Scrub the tweets of urls, hash tags, emojis, RTs, and other twitter handles.
BERT only analyzes sentence-like text.

These other features could be used as engineered features...
""" 


def clean_tweet_text(in_text):
    
    try:
        
        # remove URLS
        # regex from https://regexr.com/36fcc
        url_re = re.compile(r'(http|ftp|https)://([\w+?\.\w+])+([a-zA-Z0-9\~\!\@\#\$\%\^\&\*\(\)_\-\=\+\\\/\?\.\:\;\'\,]*)?')
        in_text = url_re.sub("",in_text)

        # remove hashtags
        hashtag_re = re.compile(r'(#\w+)')
        in_text = hashtag_re.sub("",in_text)

        # remove screen name references
        screen_name_re = re.compile(r'(@\w+)')
        in_text = screen_name_re.sub("", in_text)

        # remove RTs
        in_text = in_text.replace("RT","")

        # strip text of extra spaces , and keep one space between each word
        in_text = " ".join(in_text.split())

        # strip text of emjoi
        # https://stackoverflow.com/questions/51217909/removing-all-emojis-from-text
        emoji_re = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U0001F1F2-\U0001F1F4"  # Macau flag
            u"\U0001F1E6-\U0001F1FF"  # flags
            u"\U0001F600-\U0001F64F"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U0001F1F2"
            u"\U0001F1F4"
            u"\U0001F620"
            u"\u200d"
            u"\u2640-\u2642"
            "]+", flags=re.UNICODE)

        in_text = emoji_re.sub("",in_text)
    
    except:
        in_text = ""
    return in_text




In [0]:

# create a new column with the filtered text
tqdm.pandas()
ru_tweets['filtered_text'] = ru_tweets['content'].progress_apply(clean_tweet_text).tolist()
gwu_tweets['filtered_text'] = gwu_tweets['text'].progress_apply(clean_tweet_text).tolist()

# remove any tweets less than 10 characters

ru_tweets= ru_tweets.loc[ru_tweets['filtered_text'].progress_apply(lambda x: len(x) >= 10)]
gwu_tweets= gwu_tweets.loc[gwu_tweets['filtered_text'].progress_apply(lambda x: len(x) >= 10)]


# Let's print a sample

print(ru_tweets.sample(10)['filtered_text'].tolist())
print(gwu_tweets.sample(10)['filtered_text'].tolist())

Define the characteristics on which we will fit a model. The settings indicated are recommended, and will work for one iteration of the twitter API pull. If you have downloaded more tweets, then the sample size may be increased.

In [0]:
#@title Selecting sample size and train/test split
sample_size = 125000 #@param {type:"integer"}
training_pct = 0.8 #@param {type:"slider", min:0, max:1, step:0.05}
proportion_ira_tweets_to_gwu_tweets = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}




In [0]:
# create a new dataframe with our annotated set
# start with 100k total examples, divided 50/50

tqdm.pandas()
random_seed = 42
number_of_ira_tweets = int(sample_size * .5)
number_of_gwu_tweets = int(sample_size * (1-.5))
train_set_size = int(sample_size * training_pct)
test_set_size = int(sample_size * (1-training_pct))

labeled_set = pd.DataFrame({'text':ru_tweets['filtered_text'].sample(number_of_ira_tweets, random_state=random_seed).tolist() + gwu_tweets['filtered_text'].sample(number_of_gwu_tweets, random_state=random_seed).tolist(), 
                            'label':[1 for x in range(0,number_of_ira_tweets)] + [0 for x in range(0,number_of_gwu_tweets)]})


# re-shuffle

labeled_set = labeled_set.sample(frac=1, random_state=random_seed)
train_labeled_set = labeled_set[:train_set_size]
test_labeled_set = labeled_set[train_set_size:]



In [0]:
labeled_set.info()

In [0]:
# Importing our BERT-specific functions

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [0]:
# Adopted from https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

train_InputExamples = train_labeled_set.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test_labeled_set.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
def create_tokenizer():
  
  """
  Reviews the type of BERT module selected earlier, 
  and create the appropriate tokenizer.
  
  A tokenizer is required for inputing text into BERT
  """
    
  do_lower_case = False
  if 'uncased' in bert_model_choice.value:
        cased_option = False
  else:
        cased_option = True
        
  vocab_file = TF_CHECKPOINT_ROOT + '/' + tf_checkpoint_sub_dir.value + 'vocab.txt'
  
  with tf.Graph().as_default():      
      return bert.tokenization.FullTokenizer(
          vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer()

In [0]:
# We'll set sequences to be at most 256 tokens long
# Note, this is not 256 words, but 256 tokens
# obviously, for tweets, this should be sufficient
MAX_SEQ_LENGTH = 256

# Convert our train and test features to InputFeatures that BERT understands.
# Save the results as a TFRecord, which will be copied to Google Cloud Storage,
# so, that the TPU can read the TFRecord directly

train_features = bert.run_classifier.file_based_convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer,'train_features.TFRecord')
test_features = bert.run_classifier.file_based_convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer,'test_features.TFRecord')

In [0]:
# Copy the generated TFRecords to the data_cache, so that the TPU may directly access

!gsutil cp train_features.TFRecord {DATA_CACHE}/train_features.TFRecord
!gsutil cp test_features.TFRecord {DATA_CACHE}/test_features.TFRecord


In [0]:
# Adopted from https://github.com/google-research/bert/blob/ffbda2a1aafe530525212d13194cc84d92ed0313/run_classifier.py#L574

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  # Since we are classifying sentences, we are using pooled, not
  # token output.
  output_layer = model.get_pooled_output()

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


In [0]:
# Adopted from https://github.com/google-research/bert/blob/ffbda2a1aafe530525212d13194cc84d92ed0313/run_classifier.py#L619

def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(values=per_example_loss)
        f1_score = tf.contrib.metrics.f1_score(label_ids,predictions)
        auc = tf.metrics.auc(label_ids,predictions)
        recall = tf.metrics.recall(label_ids,predictions)
        precision = tf.metrics.precision(label_ids,predictions) 
        true_pos = tf.metrics.true_positives(label_ids,predictions)
        true_neg = tf.metrics.true_negatives(label_ids,predictions)   
        false_pos = tf.metrics.false_positives(label_ids,predictions)  
        false_neg = tf.metrics.false_negatives(label_ids,predictions)

        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
            "eval_f1_score": f1_score,
            "eval_auc": auc,
            "eval_recall": recall,
            "eval_precision": precision,
            "eval_true_pos": true_pos,
            "eval_true_neg": true_neg,
            "eval_false_pos": false_pos,
            "eval_false_neg": false_neg,
        }

      eval_metrics = (metric_fn,
                      [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities},
          scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [0]:
# These default settings are recommended. Feel feel to change to experiment.

TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
PREDICT_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 6.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 5000
SAVE_SUMMARY_STEPS = 2500
NUM_TPU_CORES = 8
OUTPUT_DIR = TF_CHECKPOINT_ROOT + '/' + tf_checkpoint_sub_dir.value
ITERATIONS_PER_LOOP = 1000
MAX_SEQ_LENGTH= 256

In [0]:
# Adopted from https://github.com/google-research/bert/blob/bee6030e31e42a9394ac567da170a89a98d2062f/run_classifier_with_tfhub.py#L180

# Create an object to represent the TPU cluster
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

# Create our configuration file
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
print(f'Number of training steps is {num_train_steps}, and number of warmup steps is {num_warmup_steps}')

In [0]:
# Adopted from https://github.com/google-research/bert/blob/bee6030e31e42a9394ac567da170a89a98d2062f/run_classifier_with_tfhub.py#L89
# Generate a model from our model builder factory  
    
model_fn = model_fn_builder(
    bert_config=  modeling.BertConfig.from_json_file(TF_CHECKPOINT_ROOT + '/' + tf_checkpoint_sub_dir.value + 'bert_config.json'),
    num_labels=len(label_list),
    init_checkpoint=TF_CHECKPOINT_ROOT + '/' + tf_checkpoint_sub_dir.value + 'bert_model.ckpt',
    learning_rate=2e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=True,
    use_one_hot_embeddings=True)

In [0]:
# Create the estimator
    
estimator = tf.contrib.tpu.TPUEstimator(
use_tpu=True,
model_fn=model_fn,
config=run_config,
train_batch_size=TRAIN_BATCH_SIZE,
eval_batch_size=EVAL_BATCH_SIZE,
predict_batch_size=PREDICT_BATCH_SIZE)


In [0]:
# Create an input function for training, using the TFRecord we generated

train_input_fn = bert.run_classifier.file_based_input_fn_builder(
        input_file=DATA_CACHE + '/train_features.TFRecord',
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=True)


In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Training is done! Let's see how accurate our model is:

In [0]:
# Now, let's evaluate to see how we've done.

test_input_fn = bert.run_classifier.file_based_input_fn_builder(
        input_file=DATA_CACHE + '/test_features.TFRecord',
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=True)


In [0]:
# The calculation for evaluation steps is separate from the calculation for training steps
eval_steps = int(len(test_InputExamples) / EVAL_BATCH_SIZE)
print(eval_steps)

In [0]:
results_dict = estimator.evaluate(input_fn=test_input_fn, steps=eval_steps)
print(results_dict)

If everything went well, you should have received at least 90 percent accuracy, recall, and precision!

In [0]:
# Let's make some predictions - for this to work, we have to create a TF Record...
# Adopted from https://github.com/google-research/bert/blob/0a0ea64a3ac1f43ed27d75278b9578708f9febcf/predicting_movie_reviews_with_bert_on_tf_hub.ipynb#L1090

def getPrediction(in_sentences):
  ret = []
  try:
        
      labels = [0, 1]
      input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
      input_features = run_classifier.file_based_convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer,'predict_features.TFRecord')
      !gsutil cp predict_features.TFRecord {DATA_CACHE}/predict_features.TFRecord
      predict_input_fn = run_classifier.file_based_input_fn_builder(input_file=DATA_CACHE + '/predict_features.TFRecord', seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
      predictions = estimator.predict(predict_input_fn)
      #ret = [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]
      for p in predictions:
            ret.append(p)
  except IndexError:
      return(ret)
  return(ret)  

Now, try your own predictions! With a batch size set at 32, for TPU prediction, make sure you are sending predictions that match the batch size.

In [0]:
pred_sentences = [
  "Sentence 1",
  "Sentence 2",
  "Sentence 3",
  "Sentence 4"
]

In [0]:
# Matching batch size
pred_sentences = pred_sentences * 8
preds = getPrediction(pred_sentences)

In [0]:
import numpy as np
pred_probs = [np.argmax(p['probabilities']) for p in preds]
final_predictions = list(zip(pred_sentences, pred_probs))
print(final_predictions)

## Adding Features to the BERT Classification Model
In the example above, text classification was accomplished through the BERT model analyzing only the filtered text of the tweet. With 125,000 features on an 80/20 train-test split,94 percent recall and 93 percent precision were observed in the test set. That's quite good!

Perhaps, we can add some aditional features using the tweet metadata to further improve the scores? This will require some feature engineering.

For this exercise, three features will be added: 1) a feature-scaled count of the number of followers, a feature-scaled count of the number of screen names followed by the respective tweeter, and third, the ratio between the number of followers and followees. 



In [0]:
#@title (Keep or change) Selecting sample size and train/test split
sample_size = 125000 #@param {type:"integer"}
training_pct = 0.8 #@param {type:"slider", min:0, max:1, step:0.05}
proportion_ira_tweets_to_gwu_tweets = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}




In [0]:
"""

# Let's go back to our original dictionary of tweets

# The relevant features for each original dataset are:

ru_tweets['followers']
ru_tweets['following']

gwu_tweets['user.followers_count']
gwu_tweets['users.friends_count']

"""

tqdm.pandas()
random_seed = 42
number_of_ira_tweets = int(sample_size * .5)
number_of_gwu_tweets = int(sample_size * (1-.5))
train_set_size = int(sample_size * training_pct)
test_set_size = int(sample_size * (1-training_pct))

extra_features = pd.DataFrame({'text':ru_tweets['filtered_text'].sample(number_of_ira_tweets, random_state=random_seed).tolist() + 
                                           gwu_tweets['filtered_text'].sample(number_of_gwu_tweets, random_state=random_seed).tolist(), 
                            'label':[1 for x in range(0,number_of_ira_tweets)] + [0 for x in range(0,number_of_gwu_tweets)],
                                          'followers':ru_tweets['followers'].sample(number_of_ira_tweets, random_state=random_seed).tolist() + 
                                           gwu_tweets['user.followers_count'].sample(number_of_gwu_tweets, random_state=random_seed).tolist(),
                                          'following': ru_tweets['followers'].sample(number_of_ira_tweets, random_state=random_seed).tolist() + 
                                           gwu_tweets['user.friends_count'].sample(number_of_gwu_tweets, random_state=random_seed).tolist()})


# re-shuffle

extra_features = extra_features.sample(frac=1, random_state=random_seed)

# let's use some feature scaling

followers_stats={
'max_values':max(extra_features.followers.tolist()),
'min_values':min(extra_features.followers.tolist()),
'avg_values':np.mean(extra_features.followers.tolist()),
'median_values':np.median(extra_features.followers.tolist())
}
print(followers_stats)

# Yikes, this distribution is heavily skewed towards zero!

following_stats={
'max_values':max(extra_features.following.tolist()),
'min_values':min(extra_features.following.tolist()),
'avg_values':np.mean(extra_features.following.tolist()),
'median_values':np.median(extra_features.following.tolist())
}

# This distribution does not look as skewed.

# let's conduct feature scaling based on the median.

def feature_scale_median(x,x_median,x_min,x_max):
    
    return((x-x_median)/(x_max-x_min))

extra_features['followers_scaled'] = extra_features['followers'].apply(feature_scale_median, 
                                                                       x_median=followers_stats['median_values'],
                                                                      x_min=followers_stats['min_values'],
                                                                      x_max=followers_stats['max_values'])


extra_features['following_scaled'] = extra_features['following'].apply(feature_scale_median, 
                                                                       x_median=following_stats['median_values'],
                                                                      x_min=following_stats['min_values'],
                                                                      x_max=following_stats['max_values'])


extra_features['following_follower_ratio'] = extra_features['following'] / extra_features['followers']

# replace any non-numbers or infinite values with zero

extra_features['following_follower_ratio'] = extra_features['following_follower_ratio'].apply(lambda x: 0 if (np.isnan(x) or np.isinf(x)) else x)
extra_features['following_follower_ratio'].describe()

# convert our three extra features into a single list

extra_features_list = list(zip(extra_features['followers_scaled'].tolist(),
                              extra_features['following_scaled'].tolist(),
                              extra_features['following_follower_ratio'].tolist()))

# convert from a list of tuples to a list of lists
extra_features_list = list(zip(extra_features['followers_scaled'].tolist(),
                              extra_features['following_scaled'].tolist(),
                              extra_features['following_follower_ratio'].tolist()))


extra_features_list = [[a,b,c] for a,b,c in extra_features_list]
extra_features['extra_features'] = extra_features_list                           


train_extra_features = extra_features[:train_set_size]
test_extra_features = extra_features[train_set_size:]

# setting a global variable for extra features. This isn't the best practice,
# but for now...

num_of_extra_features = 3


Since we are creating a generating a new model, and because we do not want to wipe the previous model, we need to create a new folder in your tf_checkpoint_root with the new model. Made sure that you've selected a BERT model in the previous cells - otherwise, this function will fail.

In [0]:
print("Select a location for the tensorflow model. Run the function two cells down to create a new folder.")
extra_features_model_folder = widgets.Dropdown(options=tf.io.gfile.listdir(TF_CHECKPOINT_ROOT), value=tf.io.gfile.listdir(TF_CHECKPOINT_ROOT)[0])
extra_features_model_folder

In [0]:
# assign the var
extra_features_model_folder = str(extra_features_model_folder.value[:-1])
print(f'Working from {extra_features_model_folder} as {type(extra_features_model_folder)}')

In [0]:
## This code wil create a new sub-folder and populate a folder with a BERT model
## **Only run if you want to create a new folder. Otherwise, select a model from above.
import urllib
cur_name = namegenerator.gen()
text_input = input(f'Enter a model name, or <ENTER> for {cur_name}:')
if text_input is '':
    text_input=cur_name
print(f'Creating folder for {text_input}')
tf.gfile.MakeDirs(TF_CHECKPOINT_ROOT + '/' + text_input)
extra_features_model_folder = text_input
bert_file_name = bert_model_choice.value.split('/')[-1]
print(bert_file_name)
urllib.request.urlretrieve(bert_model_choice.value, bert_file_name )
print("downloaded")
!ls
!unzip -o {bert_file_name}
!cd {bert_file_name.split('.')[0]};gsutil cp * {TF_CHECKPOINT_ROOT + '/' + extra_features_model_folder}



In [0]:
# Now, we're going to lightly edit the original code from the BERT repo
# to add our extra features

# From https://github.com/google-research/bert/blob/ffbda2a1aafe530525212d13194cc84d92ed0313/run_classifier.py#L161


class InputExampleExtraFeatures(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None,extra_features=None):
    """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
      extra_features: 1-D numpy array of extra features
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label
    self.extra_features = extra_features



In [0]:
# From https://github.com/google-research/bert/blob/ffbda2a1aafe530525212d13194cc84d92ed0313/run_classifier.py#L479

def file_based_convert_examples_to_features_with_extra_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  writer = tf.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example_extra_features(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f
    
    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f
    

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    features["label_ids"] = create_int_feature([feature.label_id])

    # Adding extra feature here
    features["extra_features"] = create_float_feature(feature.extra_features)
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()

In [0]:
# From https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

train_InputExamples_extra_features = train_extra_features.apply(lambda x: InputExampleExtraFeatures(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN], # adding extra features
                                                                   extra_features = x['extra_features']), axis = 1)

test_InputExamples_extra_features = test_extra_features.apply(lambda x: InputExampleExtraFeatures(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN], # adding extra features
                                                                   extra_features = x['extra_features']), axis = 1)

In [0]:
# From https://github.com/google-research/bert/blob/f39e881b169b9d53bea03d2d341b31707a6c052b/run_classifier.py#L161

import collections

class PaddingInputExample_ExtraFeatures(object):
    pass

class InputFeatures_ExtraFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               extra_features,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.extra_features = extra_features
    self.is_real_example = is_real_example

def convert_single_example_extra_features(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""

  if isinstance(example, PaddingInputExample_ExtraFeatures):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        # if this is padding, insert 3 zeros for extra features
        extra_features=[0] * num_of_extra_features,
        is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if ex_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("label: %s (id = %d)" % (example.label, label_id))
    tf.logging.info("extra_features: %s" % " ".join([str(x) for x in example.extra_features]))
    

  feature = InputFeatures_ExtraFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id,
      extra_features=example.extra_features,
      is_real_example=True)
  return feature


In [0]:
MAX_SEQ_LENGTH = 256
label_list = [0,1]
train_features = file_based_convert_examples_to_features_with_extra_features(train_InputExamples_extra_features, label_list, MAX_SEQ_LENGTH, tokenizer,'train_extra_features.TFRecord')
test_features = file_based_convert_examples_to_features_with_extra_features(test_InputExamples_extra_features, label_list, MAX_SEQ_LENGTH, tokenizer,'test_extra_features.TFRecord')

In [0]:

!gsutil cp train_extra_features.TFRecord {DATA_CACHE}/train_extra_features.TFRecord
!gsutil cp test_extra_features.TFRecord {DATA_CACHE}/test_extra_features.TFRecord
# Uncomment these lines if you want to copy from data_cache to your local
# 
#!gsutil cp {DATA_CACHE}/train_extra_features.TFRecord train_extra_features.TFRecord 
#!gsutil cp {DATA_CACHE}/test_extra_features.TFRecord test_extra_features.TFRecord
#

The original create_model function is altered here to add the extra features. In this case, the extra features are concatenated to the pooled output vector, and the weights for this vector are then exposed as trainable.

The same optimizer for the BERT model is then utilized for the entire set of trainable variables - including the variables that were just created. In this case, the model is being both fine-tuned and trained with these three extra variables. 

An alternative approach could have also included fine-tuning the model and the extracting the fine-tuned vector before the binary classification layer, and then running a separate model on the concatenated output features + extra features.

Additional experimentation is needed to determine the best approach. However, for immediate purposes, editing the existing create_model code to accomodate the new extra features, and then allowing the model to train on this amended model was the most straightforward approach.

In [0]:
# Adopted from https://github.com/google-research/bert/blob/f39e881b169b9d53bea03d2d341b31707a6c052b/run_classifier.py#L574

def create_model_extra_features(bert_config, is_training, input_ids, input_mask, segment_ids,extra_features,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)


  output_layer = model.get_pooled_output()
  # Here, we make alterations to add the extra features
  output_layer_extra_features = tf.concat([output_layer,tf.convert_to_tensor(extra_features, dtype=tf.float32)],axis=1)  
    
  hidden_size = output_layer_extra_features.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer_extra_features = tf.nn.dropout(output_layer_extra_features, keep_prob=0.9)

    logits = tf.matmul(output_layer_extra_features, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


In [0]:
#Adopted from https://github.com/google-research/bert/blob/f39e881b169b9d53bea03d2d341b31707a6c052b/run_classifier.py#L509

def file_based_input_fn_builder_extra_features(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "extra_features": tf.FixedLenFeature([num_of_extra_features], tf.float32), #Adding extra features
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t
    
    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn


In [0]:
# Adopted from https://github.com/google-research/bert/blob/f39e881b169b9d53bea03d2d341b31707a6c052b/run_classifier.py#L619

def model_fn_builder_extra_features(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    extra_features = features["extra_features"] # Adding extra features

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model_extra_features(
        bert_config, is_training, input_ids, input_mask, segment_ids, extra_features, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(values=per_example_loss)
        f1_score = tf.contrib.metrics.f1_score(label_ids,predictions)
        auc = tf.metrics.auc(label_ids,predictions)
        recall = tf.metrics.recall(label_ids,predictions)
        precision = tf.metrics.precision(label_ids,predictions) 
        true_pos = tf.metrics.true_positives(label_ids,predictions)
        true_neg = tf.metrics.true_negatives(label_ids,predictions)   
        false_pos = tf.metrics.false_positives(label_ids,predictions)  
        false_neg = tf.metrics.false_negatives(label_ids,predictions)

        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
            "eval_f1_score": f1_score,
            "eval_auc": auc,
            "eval_recall": recall,
            "eval_precision": precision,
            "eval_true_pos": true_pos,
            "eval_true_neg": true_neg,
            "eval_false_pos": false_pos,
            "eval_false_neg": false_neg,
        }

      eval_metrics = (metric_fn,
                      [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities},
          scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_InputExamples_extra_features) / TRAIN_BATCH_SIZE * 6)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
print(f'Number of training steps is {num_train_steps}, and number of warmup steps is {num_warmup_steps}')

model_fn_extra_features = model_fn_builder_extra_features(
  bert_config=  modeling.BertConfig.from_json_file(TF_CHECKPOINT_ROOT + '/' + extra_features_model_folder + '/bert_config.json'),
  num_labels=len(label_list),
  init_checkpoint=TF_CHECKPOINT_ROOT + '/' + extra_features_model_folder + '/bert_model.ckpt',
  learning_rate=2e-5,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True)

In [0]:
run_config_extra_features = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=TF_CHECKPOINT_ROOT + '/' + extra_features_model_folder,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

In [0]:

estimator_extra_features = tf.contrib.tpu.TPUEstimator(
use_tpu=True,
model_fn=model_fn_extra_features,
config=run_config_extra_features,
train_batch_size=TRAIN_BATCH_SIZE,
eval_batch_size=EVAL_BATCH_SIZE,
predict_batch_size=PREDICT_BATCH_SIZE)


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.

train_input_fn_extra_features = file_based_input_fn_builder_extra_features(
        input_file=DATA_CACHE + '/train_extra_features.TFRecord',
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=True)


In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator_extra_features.train(input_fn=train_input_fn_extra_features, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

In [0]:
# Finally, create a test input function for evaluation

test_input_fn_extra_features = file_based_input_fn_builder_extra_features(
        input_file=DATA_CACHE + '/test_extra_features.TFRecord',
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=True)

In [0]:
eval_steps = int(len(test_InputExamples_extra_features) / EVAL_BATCH_SIZE)
print(eval_steps)

In [0]:
results_dict_extra_features = estimator_extra_features.evaluate(input_fn=test_input_fn_extra_features, steps=eval_steps)


In [0]:
print(results_dict_extra_features)

## All done!

*What's next?*

- This notebook is obviously a bit unwieldy, so converting the code into class modules would be a useful exercise.

- The three added features did not seem to have much effect on precision or recall. Perhaps new features could be evaluated - or, perhaps the BERT language embeddings provide enough information to fit a model.

- More robust testing on out-of-sample observations is needed to determine if the model has overfit the training set. The test set evaluation appears to indicate that overfitting is not a concern; however, a more robust test is needed.

- The limitations of the Colab environment limit the ability to ingest and train on a larger training and test set. A model could be created outside the Colab environment on a much larger training/test set, and then wrapped as a RESTful API for real-time predictions.

- Who knows!
